In [1]:
import time

start_time = time.time()

### Importações das Bibs Necessárias


In [2]:
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio import SeqIO
from Bio.Application import ApplicationError
import tempfile
import os
from io import StringIO

### Leitura das Sequências

Recebe o path do arquivo fasta e retorna array com sequências


In [3]:
def read_genome_sequences(file_genome_sequences):
    multiple_genomes = []

    with open(file_genome_sequences) as handle:
        for values in SeqIO.FastaIO.SimpleFastaParser(handle):
            multiple_genomes.append(values)

    # number_sequences = len(multiple_genomes)
    # print('number of sequences in file',
    #       file_genome_sequences, ' = ', number_sequences)

    return multiple_genomes

### Método para Encontrar as Posições do Gene no Genoma


In [4]:
def fetch_gene_position(gene_path, genome_path):
    try:
        # Executar o BLAST para buscar a posição do gene no genoma
        comando_blast = NcbiblastnCommandline(
            query=gene_path, subject=genome_path, outfmt=5)
        stdout, stderr = comando_blast()

        # Analisar o resultado XML do BLAST
        resultado_blast = NCBIXML.read(StringIO(stdout))
        if len(resultado_blast.alignments) == 0:
            print('O gene não foi encontrado no genoma.')
            return

        alinhamento = resultado_blast.alignments[0]
        hsp = alinhamento.hsps[0]

        gene_positions = []
        gene_positions.append(hsp.sbjct_start - 1)
        gene_positions.append(hsp.sbjct_end - 1)

        return gene_positions

    except ApplicationError as e:
        print('Ocorreu um erro durante a busca da posição do gene:')
        print(e)
        return None

    except TypeError:
        print('Ocorreu um erro de tipo durante a busca da posição do gene.')
        return None

### Main


In [5]:
# Sequência do gene spike
reference_gene_file_path = '/home/m_souza/TCC/Tcc_Implementacao/Mauricio_TCC_UnEB/03_pipeline_blast/refSpike.fasta'
# Path do Dataset dos Genomas
root_directory = '/home/sharedFolder/dataset_sequences/sequencias_por_lineage'
# root_directory = '/home/maume/UnEB/TCC/Dataset/sequencias_teste'
# Nome do arquivo de saída com Genes Spike
gene_file_name = 'gene_spike.fasta'

# Variáveis para acompanhar o número de arquivos, sequencias e genes não processados
files_processed = 0
sequences_processed = 0
genes_not_found = 0

# Percorrer todas as pastas e arquivos
for root_folder, folders, files in os.walk(root_directory):
    for file in files:
        genome_file_path = os.path.join(root_folder, file)
        if genome_file_path.endswith('_unique_sequences.fasta'):
            files_processed += 1
            print(f'Processando {files_processed}º arquivo: {genome_file_path}')
            genome_sequences = read_genome_sequences(genome_file_path)

            for genome_sequence in genome_sequences:

                sequences_processed += 1

              # Criar um arquivo temporário para armazenar a sequência de DNA do genoma
                temp_file = tempfile.NamedTemporaryFile(mode='w', delete=False)
                temp_file.write(f'>{genome_sequence[0]}\n{genome_sequence[1]}')

                # Caminho para o arquivo temporário do genoma
                genoma_temp_file = temp_file.name

                # Pega as Posições Inicial e Final do Gene no Genoma e Pega a Substring
                positions = fetch_gene_position(
                    reference_gene_file_path, genoma_temp_file)

                if positions is not None:
                    gene_start_position, gene_final_position = positions
                    gene_sequence = genome_sequence[1][gene_start_position:gene_final_position + 1]
                    gene_file_path = os.path.join(root_folder, gene_file_name)

                    with open(gene_file_path, 'a') as gene_file:
                        gene_file.write(
                            f'>spike(pos.ini:{gene_start_position} pos.fin:{gene_final_position}){genome_sequence[0]}\n{gene_sequence}\n')

                    # Remover o arquivo temporário
                    os.remove(genoma_temp_file)

                    temp_file.close()
                else:
                    genes_not_found += 1
                    print(f'Sequência não processada: {genome_sequence[0]}')

print('Busca Completa Concluída.')

Processando 1º arquivo: /home/sharedFolder/dataset_sequences/sequencias_por_lineage/BQ.1.22/_unique_sequences.fasta
Processando 2º arquivo: /home/sharedFolder/dataset_sequences/sequencias_por_lineage/AY.46.2/_unique_sequences.fasta
Processando 3º arquivo: /home/sharedFolder/dataset_sequences/sequencias_por_lineage/B.1.429/_unique_sequences.fasta
Processando 4º arquivo: /home/sharedFolder/dataset_sequences/sequencias_por_lineage/AY.111/_unique_sequences.fasta
Processando 5º arquivo: /home/sharedFolder/dataset_sequences/sequencias_por_lineage/XN/_unique_sequences.fasta
Processando 6º arquivo: /home/sharedFolder/dataset_sequences/sequencias_por_lineage/AY.27/_unique_sequences.fasta
Processando 7º arquivo: /home/sharedFolder/dataset_sequences/sequencias_por_lineage/BE.3/_unique_sequences.fasta
Processando 8º arquivo: /home/sharedFolder/dataset_sequences/sequencias_por_lineage/AY.7/_unique_sequences.fasta
Processando 9º arquivo: /home/sharedFolder/dataset_sequences/sequencias_por_lineage/BA

In [6]:
from datetime import datetime

end_time = time.time()

elapsed_time = end_time - start_time
current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

### Salvando Informações Sobra a Execução


In [8]:
import platform
import psutil

machine_info = platform.uname()
memory_info = psutil.virtual_memory()

path_directory = '/home/m_souza/TCC/Tcc_Implementacao/Mauricio_TCC_UnEB/03_pipeline_blast'

content = '============================================================\n'
content += 'Informações Sobre a Última Execução:\n\n'
content += f'- Quantidade de Variantes Analisadas: {files_processed}\n'
content += f'- Quantidade de Genomas Analisados: {sequences_processed}\n'
content += f'- Quantidade de Genomas em que o Gene não foi Encontrado: {genes_not_found}\n'
content += f'- Data e Hora da Execução: {current_datetime}\n'
content += f'- Tempo Total de Execução: {round(elapsed_time, 2)} Segundos\n'
content += f'- Caminho do Dataset Utilizado: {root_directory}\n'
content += f'\nInformações Sobre a Máquina Utilizada\n\n'
content += f'- S.O.: Ubuntu 22.04.2 LTS x86_64\n'
content += f'- Kernel: {machine_info.release}\n'
content += f'- CPU: Intel i7-6500U (4) @ 3.100GHz {machine_info.processor}\n'
content += f'- Total de Memória: {round(memory_info.total / (1024 ** 3), 2)}GB\n'


output_file_name = 'execution_information.txt'
output_file_path = os.path.join(path_directory, output_file_name)

with open(output_file_path, 'a') as output_file:
    output_file.write(content)

print(f'Information saved to file: {output_file_path}')

Information saved to file: /home/m_souza/TCC/Tcc_Implementacao/Mauricio_TCC_UnEB/03_pipeline_blast/execution_information.txt
